# 1. Imports and reading data

In [1]:
%%time
import time
import re
import pandas as pd
import numpy as np
import umap
import string
import hdbscan
import spacy
import gensim
import sklearn

from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])
stop = stopwords.words('english')
model=SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
kw_extractor = KeyBERT(model=model)

%config Completer.use_jedi = False

Wall time: 34 s


In [2]:
# xlm-r-distilroberta-base-paraphrase-v1
# distilbert-base-nli-mean-tokens

In [3]:
start = time.time()

In [4]:
df = pd.read_csv('octopus_energy_tweets.csv')
df.rename(columns={"tweet":"comments"},inplace=True)

# 2. Preprocessing

In [5]:
#This takes each sentence as inputs and returns list of clean tokens for that sentence
def TextCleaner(doc):
    doc = re.sub("[,.']", "", doc)
    doc = [token for token in doc.split(' ') if len(re.sub("[a-zA-Z]", "", token)) <= 0]
    doc = nlp(" ".join(doc))
    taglist = ['RB','RBR', 'RBS','JJR','JJ','JJS','NN','NNS','VB','VBG','VBP','VBN']
    poslist = ['ADJ','ADV','NOUN','VERB']
    doc = [token.lemma_.lower() for token in doc if token.tag_ in taglist and token.pos_ in poslist]
    doc = [token for token in doc if not token in stop and len(token)>2]
    return doc

In [6]:
# This function converts a list of clean documents to trigrams 
def TextNGram(doc,ngrams = 2):
    bigram = gensim.models.Phrases(doc , min_count=5, threshold=40) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    if ngrams == 2:
        doc = [bigram_mod[sent] for sent in doc]
        return doc
    elif ngrams == 3:
        trigram = gensim.models.Phrases(bigram[doc], threshold=40)
        trigram_mod = gensim.models.phrases.Phraser(trigram)
        doc = [trigram_mod[bigram_mod[sent]] for sent in doc]
        return doc
    else:
        return doc
        

In [7]:
%%time
#First 
df['Processed'] = [TextCleaner(f) for f in df['comments']]
#Second
df['Processed_ngram'] = TextNGram(df['Processed'])

Wall time: 2.38 s


In [8]:
data = pd.Series(df['Processed_ngram'].apply(lambda x: " ".join(x) if len(x) > 4 else np.nan))

In [9]:
data.dropna(axis=0, inplace=True)
data.reset_index(drop=True,inplace=True)

In [10]:
list_data = data.unique().tolist()

# 3. Model, Dimension Reduction and Saving Model

##     a. BERT Model

In [11]:
%%time
embeddings = model.encode(list_data, show_progress_bar=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Wall time: 11.3 s


## b. UMAP Dimension Reduction

In [12]:
embedding_norm = sklearn.preprocessing.normalize(embeddings, norm='l2')

In [13]:
%%time
best_model = umap.UMAP(n_components=5,min_dist=0.0).fit_transform(embedding_norm)

# #n_neighbors=10, min_dist=0.0, 

Wall time: 9.41 s


# 4. Clustering and Reducing Clusters with Cosine-Similarity

## a. HDBSCAN Clustering

In [14]:
%%time
cluster = KMeans(n_clusters=4, init="k-means++").fit_predict(best_model)

Wall time: 50 ms


In [15]:
%%time
docs = pd.DataFrame(list_data,columns=["comments"])
docs["cluster"] = cluster
labeled_docs = docs.groupby(["cluster"], as_index=False).agg({"comments": " ".join})
array_text = labeled_docs.comments.tolist()

Wall time: 6 ms


In [16]:
%%time
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(array_text[j], top_n=10)
    print(f'\033[1m   Cluster {j+1}: \033[0m')
    print([word for word, degree in keywords])

   Cluster 1: 
['customer_service', 'hourly', 'repeatedly', 'waiting', 'deadline', 'line', 'phone', 'reset', 'send', 'respond']
   Cluster 2: 
['workforce', 'employee', 'staff', 'employer', 'corporate', 'employment', 'organizational', 'job', 'evolve', 'personnel']
   Cluster 3: 
['diesel', 'petrol', 'fuel', 'electrification', 'battery', 'signal', 'install', 'switch', 'adjust', 'radio']
   Cluster 4: 
['break', 'fix', 'repair', 'deliver', 'delivery', 'restore', 'change', 'drive', 'referral', 'fixable']
Wall time: 16.3 s


In [17]:
end = time.time()
print(end - start)

39.59368443489075
